In [ ]:
#Link the query dataset and the smell dataset with the refactoring dataset
#ref_id, files, lines
#line level co-occurence...query..smells
#method level co-occurene...query..smells (1,0)
import pandas as pd

In [ ]:
#import the refactoring dataset
ref=pd.read_csv("../data/refactorings.csv",sep="$")
#import the query dataset
query=pd.read_csv("RQ4-queries.csv", sep="?")
#import the smells dataset
smell=pd.read_csv("RQ4-smells.csv")

In [ ]:
dac_ref=ref[ref["is_dac"]==True] #we only need data-access refactorings
del ref

In [96]:
len(dac_ref.index)

18892

In [44]:
ref_lines=[]
for l in dac_ref.head(1)['lines'].tolist():
    t=eval(l)
    for tup in t:
        ref_lines.extend(list(set(list(tup))))
ref_lines

[53]

In [85]:
def is_line_matched(ref_line, target_line):
    for l in ref_line:
        if l in target_line:
            return True
    return False

In [97]:
#write the function that peforms the match at line level
rq4_dataset_line=[]
def match_by_line(row):
    d={}
    d["ref_id"]=row["ref_id"]
    d["ref_type"]=row["ref_type"]
    d["file_name"]=row["file_name"]
    fileName=eval(row["file_name"])
    #search for match with query
    
    hits=query[(query["sha"]==row["sha"]) & (query["className"].isin(fileName))]
    query_lines=[]
    for l in hits['lines'].tolist():
        query_lines.extend(eval(l))
    ref_lines=[]
    for l in eval(row['lines']):
        ref_lines.extend(list(set(list(tup))))
    if is_line_matched(ref_lines, query_lines):
        d["query"]= 1
    else:
        d["query"]=0
    #FEAR_OF_THE_UNKNOWN
    sf_hits= smell[(smell['sha']==row["sha"]) & (smell['className'].isin(fileName)) & (smell["kind"]=="FEAR_OF_THE_UNKNOWN")]
    sf_lines=[]
    for l in sf_hits['line'].tolist():
        sf_lines.append(l)
    if is_line_matched(ref_lines, sf_lines):
        d["FEAR_OF_THE_UNKNOWN"]=1
    else:
        d["FEAR_OF_THE_UNKNOWN"]=0
    si_hits= smell[(smell['sha']==row["sha"]) & (smell['className'].isin(fileName)) & (smell["kind"]=="IMPLICIT_COLUMNS")]
    #IMPLICIT_COLUMNS
    si_lines=[]
    for l in si_hits['line'].tolist():
        si_lines.append(l)
    if is_line_matched(ref_lines, si_lines):
        d["IMPLICIT_COLUMNS"]=1
    else:
        d["IMPLICIT_COLUMNS"]=0
    rq4_dataset_line.append(d)
    print("Index:{}/{}".format(row.name, len(dac_ref.index)))
    return True

In [ ]:
#apply the function
dac_ref["line_matched"]=dac_ref.apply(match_by_line, axis=1)

In [100]:
rq4_line=pd.DataFrame(rq4_dataset_line)
rq4_line

,ref_id,ref_type,file_name,query,FEAR_OF_THE_UNKNOWN,IMPLICIT_COLUMNS
0,oltpbench-d97a34c952f90de191d76ab5efaf32860d58...,Encapsulate Attribute,['src/com/oltpbenchmark/api/Loader.java'],0,0,0
1,oltpbench-d77ff02e21a0b56e74ad35bb8cd53e9a5ea4...,Extract Subclass,['src/com/oltpbenchmark/api/collectors/NoopCol...,0,0,0
2,oltpbench-35114287311ebd257fa4cdf440d754f42016...,Add Parameter,['src/com/oltpbenchmark/benchmarks/tpch/TPCHLo...,0,0,0
3,oltpbench-35114287311ebd257fa4cdf440d754f42016...,Add Parameter,['src/com/oltpbenchmark/benchmarks/tpch/TPCHLo...,0,0,0
4,oltpbench-35114287311ebd257fa4cdf440d754f42016...,Add Parameter,['src/com/oltpbenchmark/benchmarks/tpch/TPCHLo...,0,0,0


In [105]:
rq4_line.to_csv("RQ4-linematch-dataset.csv", index=False)

In [106]:
#Now lets do the method level matchs
dac_ref.head()

,repo,sha,ref_id,ref_type,key,ref_desc,file_name,method_lines,lines,is_dac,line_matched
3,oltpbench,d97a34c952f90de191d76ab5efaf32860d58b1f7,oltpbench-d97a34c952f90de191d76ab5efaf32860d58...,Encapsulate Attribute,b'wRYepah0BkPqrFALJFQ9h4sNbEJfeQfzxaTojPNF_cM=',b'gAAAAABg7M2-K9VVgaBV-FC3mUMD5lPCt0CXN-y3L_dU...,['src/com/oltpbenchmark/api/Loader.java'],"[(81, 83)]","[(53, 53)]",True,True
13,oltpbench,d77ff02e21a0b56e74ad35bb8cd53e9a5ea44194,oltpbench-d77ff02e21a0b56e74ad35bb8cd53e9a5ea4...,Extract Subclass,b'wRYepah0BkPqrFALJFQ9h4sNbEJfeQfzxaTojPNF_cM=',b'gAAAAABg7M2-AqHAmSzDYOsyIa_mRrYWxgexhj7tPYiI...,['src/com/oltpbenchmark/api/collectors/NoopCol...,"[(49, 52), (54, 57)]","[(3, 19)]",True,True
16,oltpbench,35114287311ebd257fa4cdf440d754f420163149,oltpbench-35114287311ebd257fa4cdf440d754f42016...,Add Parameter,b'wRYepah0BkPqrFALJFQ9h4sNbEJfeQfzxaTojPNF_cM=',b'gAAAAABg7M2-tGqlBcr8_PEET-rWjq-D-qChKq45XidQ...,['src/com/oltpbenchmark/benchmarks/tpch/TPCHLo...,"[(276, 278)]","[(276, 276)]",True,True
17,oltpbench,35114287311ebd257fa4cdf440d754f420163149,oltpbench-35114287311ebd257fa4cdf440d754f42016...,Add Parameter,b'wRYepah0BkPqrFALJFQ9h4sNbEJfeQfzxaTojPNF_cM=',b'gAAAAABg7M2-wF1_jGgMdb9YepEgKoozsxozQ1XB2PV-...,['src/com/oltpbenchmark/benchmarks/tpch/TPCHLo...,"[(280, 282)]","[(280, 280)]",True,True
18,oltpbench,35114287311ebd257fa4cdf440d754f420163149,oltpbench-35114287311ebd257fa4cdf440d754f42016...,Add Parameter,b'wRYepah0BkPqrFALJFQ9h4sNbEJfeQfzxaTojPNF_cM=',b'gAAAAABg7M2-1RavuhnNt1to3cf7DhrBxzaGT9Xp5OUD...,['src/com/oltpbenchmark/benchmarks/tpch/TPCHLo...,"[(284, 286)]","[(284, 284)]",True,True


In [115]:
def is_method_matched(ref_method_lines, target_lines):
    for tup in ref_method_lines:
        line_min=tup[0]
        line_max=tup[1]
        for l in target_lines:
            if line_min <= l <= line_max:
                return True
    return False

In [116]:
#write the function that peforms the match at method level
rq4_dataset_method=[]
def match_by_method(row):
    d={}
    d["ref_id"]=row["ref_id"]
    d["ref_type"]=row["ref_type"]
    d["file_name"]=row["file_name"]
    fileName=eval(row["file_name"])
    #search for match with query
    
    hits=query[(query["sha"]==row["sha"]) & (query["className"].isin(fileName))]
    query_lines=[]
    for l in hits['lines'].tolist():
        query_lines.extend(eval(l))
    ref_lines=eval(row['method_lines'])
    if is_method_matched(ref_lines, query_lines):
        d["query"]= 1
    else:
        d["query"]=0
    #FEAR_OF_THE_UNKNOWN
    sf_hits= smell[(smell['sha']==row["sha"]) & (smell['className'].isin(fileName)) & (smell["kind"]=="FEAR_OF_THE_UNKNOWN")]
    sf_lines=[]
    for l in sf_hits['line'].tolist():
        sf_lines.append(l)
    if is_method_matched(ref_lines, sf_lines):
        d["FEAR_OF_THE_UNKNOWN"]=1
    else:
        d["FEAR_OF_THE_UNKNOWN"]=0
    si_hits= smell[(smell['sha']==row["sha"]) & (smell['className'].isin(fileName)) & (smell["kind"]=="IMPLICIT_COLUMNS")]
    #IMPLICIT_COLUMNS
    si_lines=[]
    for l in si_hits['line'].tolist():
        si_lines.append(l)
    if is_method_matched(ref_lines, si_lines):
        d["IMPLICIT_COLUMNS"]=1
    else:
        d["IMPLICIT_COLUMNS"]=0
    rq4_dataset_method.append(d)
    print("Index:{}/{}".format(row.name, len(dac_ref.index)))
    return True

In [ ]:
#apply the function
dac_ref["method_matched"]=dac_ref.apply(match_by_method, axis=1)

In [118]:
rq4_method=pd.DataFrame(rq4_dataset_method)
rq4_method.head()

,ref_id,ref_type,file_name,query,FEAR_OF_THE_UNKNOWN,IMPLICIT_COLUMNS
0,oltpbench-d97a34c952f90de191d76ab5efaf32860d58...,Encapsulate Attribute,['src/com/oltpbenchmark/api/Loader.java'],0,0,0
1,oltpbench-d77ff02e21a0b56e74ad35bb8cd53e9a5ea4...,Extract Subclass,['src/com/oltpbenchmark/api/collectors/NoopCol...,1,0,0
2,oltpbench-35114287311ebd257fa4cdf440d754f42016...,Add Parameter,['src/com/oltpbenchmark/benchmarks/tpch/TPCHLo...,0,0,0
3,oltpbench-35114287311ebd257fa4cdf440d754f42016...,Add Parameter,['src/com/oltpbenchmark/benchmarks/tpch/TPCHLo...,0,0,0
4,oltpbench-35114287311ebd257fa4cdf440d754f42016...,Add Parameter,['src/com/oltpbenchmark/benchmarks/tpch/TPCHLo...,0,0,0


In [122]:
rq4_method.to_csv("RQ4-methodmatch-dataset.csv", index=False)

In [126]:
#percentage calculation refs vs sql, line level query
total_refs= len(dac_ref.index)
sql_refs= len(rq4_line[rq4_line["query"]==1].index)
percentage= (sql_refs*100)/total_refs
print("Number of data-access refactoring instances: {}".format(total_refs))
len(rq4_line[rq4_line["query"]==1])
print("Number of data-access refactoring instances that touch query: {}, pecentage: {}".format(sql_refs,percentage))

Number of data-access refactoring instances: 18892
Number of data-access refactoring instances that touch query: 86, pecentage: 0.4552191403768791


In [127]:
sql_refs_l=rq4_line[rq4_line["query"]==1]
sql_refs_l.head()

,ref_id,ref_type,file_name,query,FEAR_OF_THE_UNKNOWN,IMPLICIT_COLUMNS
74,oltpbench-684986a1fac10da9270aadc3d93cda0d5eea...,Add Parameter,['src/com/oltpbenchmark/benchmarks/resourcestr...,1,0,0
75,oltpbench-684986a1fac10da9270aadc3d93cda0d5eea...,Add Parameter,['src/com/oltpbenchmark/benchmarks/resourcestr...,1,0,0
76,oltpbench-684986a1fac10da9270aadc3d93cda0d5eea...,Add Parameter,['src/com/oltpbenchmark/benchmarks/resourcestr...,1,0,0
77,oltpbench-684986a1fac10da9270aadc3d93cda0d5eea...,Remove Variable Modifier,['src/com/oltpbenchmark/benchmarks/resourcestr...,1,0,0
78,oltpbench-684986a1fac10da9270aadc3d93cda0d5eea...,Parameterize Variable,['src/com/oltpbenchmark/benchmarks/resourcestr...,1,0,0


In [130]:
sql_refs_l.groupby(by="ref_type").size().sort_values(ascending=False)

ref_type
Change Return Type          17
Add Parameter               12
Extract Method              10
Change Variable Type         7
Move Source Folder           5
Extract Variable             5
Inline Method                4
Extract And Move Method      4
Add Method Annotation        3
Remove Variable Modifier     2
Add Class Annotation         2
Parameterize Variable        2
Move Class                   2
Inline Variable              2
Rename Variable              1
Change Package               1
Change Attribute Type        1
Extract Class                1
Change Parameter Type        1
Encapsulate Attribute        1
Rename Attribute             1
Remove Class Annotation      1
Add Attribute Modifier       1
dtype: int64

In [131]:
17*100/86

19.767441860465116

In [132]:
12*100/86

13.953488372093023

In [133]:
10*100/86

11.627906976744185

In [134]:
7*100/86

8.13953488372093

In [135]:
#percentage calculation refs vs sql, method level
total_refs= len(dac_ref.index)
sql_refs= len(rq4_method[rq4_method["query"]==1].index)
percentage= (sql_refs*100)/total_refs
print("Number of data-access refactoring instances: {}".format(total_refs))
len(rq4_line[rq4_line["query"]==1])
print("Number of data-access refactoring instances that touch query: {}, pecentage: {}".format(sql_refs,percentage))

Number of data-access refactoring instances: 18892
Number of data-access refactoring instances that touch query: 5607, pecentage: 29.67922930340885


In [136]:
sql_refs_m=rq4_method[rq4_method["query"]==1]
sql_refs_m.groupby(by="ref_type").size().sort_values(ascending=False)

ref_type
Rename Variable                    821
Change Variable Type               603
Add Parameter                      409
Change Parameter Type              355
Extract Method                     350
Extract Variable                   320
Rename Parameter                   301
Add Parameter Modifier             273
Change Thrown Exception Type       265
Rename Method                      230
Change Return Type                 211
Inline Variable                    175
Extract And Move Method            125
Add Thrown Exception Type          118
Add Method Annotation              116
Remove Parameter Modifier          115
Add Variable Modifier               98
Remove Method Annotation            89
Change Method Access Modifier       87
Remove Parameter                    83
Remove Thrown Exception Type        76
Move And Inline Method              75
Remove Method Modifier              54
Inline Method                       53
Move Method                         50
Pull Up Method  

In [137]:
821*100/5607

14.642411271624756

In [138]:
603*100/5607

10.754414125200642

In [139]:
409*100/5607

7.294453361869092

In [140]:
355*100/5607

6.3313714999108255

In [141]:
6*100/5607

0.10700909577314072

In [143]:
rq4_line.head()

,ref_id,ref_type,file_name,query,FEAR_OF_THE_UNKNOWN,IMPLICIT_COLUMNS
0,oltpbench-d97a34c952f90de191d76ab5efaf32860d58...,Encapsulate Attribute,['src/com/oltpbenchmark/api/Loader.java'],0,0,0
1,oltpbench-d77ff02e21a0b56e74ad35bb8cd53e9a5ea4...,Extract Subclass,['src/com/oltpbenchmark/api/collectors/NoopCol...,0,0,0
2,oltpbench-35114287311ebd257fa4cdf440d754f42016...,Add Parameter,['src/com/oltpbenchmark/benchmarks/tpch/TPCHLo...,0,0,0
3,oltpbench-35114287311ebd257fa4cdf440d754f42016...,Add Parameter,['src/com/oltpbenchmark/benchmarks/tpch/TPCHLo...,0,0,0
4,oltpbench-35114287311ebd257fa4cdf440d754f42016...,Add Parameter,['src/com/oltpbenchmark/benchmarks/tpch/TPCHLo...,0,0,0


In [146]:
rq4_line.head()

,ref_id,ref_type,file_name,query,FEAR_OF_THE_UNKNOWN,IMPLICIT_COLUMNS,smell
0,oltpbench-d97a34c952f90de191d76ab5efaf32860d58...,Encapsulate Attribute,['src/com/oltpbenchmark/api/Loader.java'],0,0,0,0
1,oltpbench-d77ff02e21a0b56e74ad35bb8cd53e9a5ea4...,Extract Subclass,['src/com/oltpbenchmark/api/collectors/NoopCol...,0,0,0,0
2,oltpbench-35114287311ebd257fa4cdf440d754f42016...,Add Parameter,['src/com/oltpbenchmark/benchmarks/tpch/TPCHLo...,0,0,0,0
3,oltpbench-35114287311ebd257fa4cdf440d754f42016...,Add Parameter,['src/com/oltpbenchmark/benchmarks/tpch/TPCHLo...,0,0,0,0
4,oltpbench-35114287311ebd257fa4cdf440d754f42016...,Add Parameter,['src/com/oltpbenchmark/benchmarks/tpch/TPCHLo...,0,0,0,0


<h3>Smells</h3>

In [157]:
#SQL smells line level
rq4_line["smell"]=rq4_line["IMPLICIT_COLUMNS"]+rq4_line["FEAR_OF_THE_UNKNOWN"]

In [158]:
#SQL smells method level
rq4_method["smell"]=rq4_method["IMPLICIT_COLUMNS"]+rq4_method["FEAR_OF_THE_UNKNOWN"]

In [161]:
#percentage calculation refs vs sql, line level
total_refs= len(dac_ref.index)
smell_refs= len(rq4_line[rq4_line["smell"]>=1].index)
percentage= (smell_refs*100)/total_refs
print("Number of data-access refactoring instances: {}".format(total_refs))
len(rq4_line[rq4_line["smell"]>=1].index)
print("Number of data-access refactoring instances that touch smell: {}, pecentage: {}".format(smell_refs,percentage))

Number of data-access refactoring instances: 18892
Number of data-access refactoring instances that touch smell: 0, pecentage: 0.0


In [160]:
#percentage calculation refs vs sql, method level
total_refs= len(dac_ref.index)
smell_refs= len(rq4_method[rq4_method["smell"]>=1].index)
percentage= (smell_refs*100)/total_refs
print("Number of data-access refactoring instances: {}".format(total_refs))
len(rq4_method[rq4_method["smell"]>=1].index)
print("Number of data-access refactoring instances that touch smell: {}, pecentage: {}".format(smell_refs,percentage))

Number of data-access refactoring instances: 18892
Number of data-access refactoring instances that touch smell: 256, pecentage: 1.3550709294939658


In [153]:
rq4_method[rq4_method["smell"]>=1]

,ref_id,ref_type,file_name,query,FEAR_OF_THE_UNKNOWN,IMPLICIT_COLUMNS,smell
1905,Applozic-Android-SDK-6d74704063e40d62e6891a33c...,Rename Method,['mobicomkit/src/main/java/com/applozic/mobico...,1,1,1,1
3888,WordPress-Android-ee8c44e9d898472f54ee31cfdc6e...,Change Return Type,['src/org/wordpress/android/datasets/ReaderBlo...,1,1,1,1
3891,WordPress-Android-ee8c44e9d898472f54ee31cfdc6e...,Inline Method,['src/org/wordpress/android/datasets/ReaderBlo...,1,1,1,1
3892,WordPress-Android-ee8c44e9d898472f54ee31cfdc6e...,Change Variable Type,['src/org/wordpress/android/datasets/ReaderBlo...,1,1,1,1
3893,WordPress-Android-ee8c44e9d898472f54ee31cfdc6e...,Change Variable Type,['src/org/wordpress/android/datasets/ReaderBlo...,1,1,1,1
3896,WordPress-Android-ee8c44e9d898472f54ee31cfdc6e...,Change Variable Type,['src/org/wordpress/android/datasets/ReaderBlo...,1,1,1,1
3897,WordPress-Android-ee8c44e9d898472f54ee31cfdc6e...,Rename Variable,['src/org/wordpress/android/datasets/ReaderBlo...,1,1,1,1


In [163]:
s=rq4_method[rq4_method["smell"]>=1]
s.groupby(by="ref_type").size().sort_values(ascending=False)

ref_type
Rename Variable                  60
Extract Method                   30
Rename Method                    26
Add Parameter                    24
Change Variable Type             21
Rename Parameter                 16
Change Return Type               15
Change Parameter Type            14
Remove Parameter                  8
Extract And Move Method           8
Inline Method                     7
Extract Variable                  6
Add Thrown Exception Type         4
Change Thrown Exception Type      4
Remove Method Annotation          3
Inline Variable                   2
Remove Method Modifier            2
Change Method Access Modifier     2
Extract Class                     1
Add Variable Modifier             1
Parameterize Variable             1
Add Method Annotation             1
dtype: int64

In [164]:
60* 100/256

23.4375

In [165]:
30*100/256

11.71875

In [166]:
26*100/256

10.15625

In [167]:
24*100/256

9.375

In [168]:
21*100/256

8.203125

In [176]:
total_smell=len(rq4_method[rq4_method["smell"]>=1].index)
ic_smell=len((rq4_method[(rq4_method["IMPLICIT_COLUMNS"]>=1) & (rq4_method["FEAR_OF_THE_UNKNOWN"]==0)].index))
print(ic_smell)
print(total_smell)

200
256


In [177]:
total_smell=len(rq4_method[rq4_method["smell"]>=1].index)
fu_smell=len((rq4_method[(rq4_method["IMPLICIT_COLUMNS"]==0) & (rq4_method["FEAR_OF_THE_UNKNOWN"]==1)].index))
print(fu_smell)
print(total_smell)

49
256


In [178]:
200*100/256

78.125

In [179]:
49*100/256

19.140625